In [1]:
import os, json, glob
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Repeated CV con (pequeña) búsqueda por fold y recolección de métricas
Mejoando modelos....

In [2]:
feats=pd.read_csv("../res/Features_extraction_Mantra.csv")

In [3]:
import numpy as np, pandas as pd, json, os
from typing import Dict, Any, Tuple, List
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, RandomizedSearchCV
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from joblib import parallel_backend, dump as joblib_dump

# ========== Utils básicos ==========
def _drop_constant_features(X: pd.DataFrame) -> pd.DataFrame:
    nunq = X.nunique(dropna=True)
    X = X.drop(columns=nunq[nunq <= 1].index.tolist(), errors='ignore')
    var0 = X.var(axis=0, skipna=True)
    return X.drop(columns=var0[var0 == 0].index.tolist(), errors='ignore')

def _select_feature_set(fullX: pd.DataFrame, feature_set: str) -> pd.DataFrame:
    fs = feature_set.lower()
    cols = []
    if fs in ("hvg","all"):
        cols += [c for c in fullX.columns if c.startswith("hvg_")]
    if fs in ("dhvg","all"):
        cols += [c for c in fullX.columns if c.startswith("dhvg_")]
    if fs in ("whvg","all"):
        cols += [c for c in fullX.columns if c.startswith("whvg_") or c.startswith("wvg_")]
    seen = set(); cols = [c for c in cols if not (c in seen or seen.add(c))]
    if not cols:
        cols = [c for c in fullX.columns if c.startswith(("hvg_","dhvg_","whvg_","wvg_"))]
    return fullX[cols].copy()

def _prep_feats_labels(feats: pd.DataFrame, feature_set: str):
    assert 'Classification' in feats.columns
    X_full = feats.drop(columns=[c for c in ['ID','Classification'] if c in feats.columns]).copy()
    X_full = X_full.apply(pd.to_numeric, errors='coerce')
    X_full = _drop_constant_features(X_full)
    X = _select_feature_set(X_full, feature_set)
    y_raw = feats['Classification'].astype(str).values
    le = LabelEncoder(); y = le.fit_transform(y_raw)
    return X, y, le

# ========== Modelos e hiperparámetros ==========
def _make_model_and_space(model:str, seed:int):
    """
    Soporta: 'lgbm' (default), 'xgb', 'rf', 'et'
    - Si XGBoost no está disponible, cae a RF.
    """
    mdl = model.lower()

    # Intentar XGBoost opcional
    _has_xgb = True
    try:
        from xgboost import XGBClassifier
    except Exception:
        _has_xgb = False

    if mdl == 'lgbm':
        try:
            from lightgbm import LGBMClassifier
        except Exception:
            mdl = 'rf'

    if mdl == 'lgbm':
        from lightgbm import LGBMClassifier
        clf = LGBMClassifier(
            objective='multiclass', n_estimators=800, class_weight='balanced',
            num_leaves=31, min_child_samples=60, min_split_gain=1e-3,
            subsample=0.8, colsample_bytree=0.8, verbose=-1, random_state=seed, device ='gpu', gpu_platform_id=0,
    gpu_device_id=0
        )
        space = {
            'clf__num_leaves': [15,31,63],
            'clf__max_depth': [-1,6,8,12],
            'clf__learning_rate': np.logspace(-2.3, -0.5, 8),
            'clf__subsample': [0.7,0.8,0.9,1.0],
            'clf__colsample_bytree': [0.7,0.8,0.9,1.0],
            'clf__min_child_samples': [40,60,80],
            'clf__reg_alpha': np.logspace(-6,-1,6),
            'clf__reg_lambda': np.logspace(-3,0,7),
            'clf__min_split_gain': [0.0,1e-4,1e-3],
        }
        return clf, space, 'lgbm'

    if mdl == 'xgb' and _has_xgb:
        from xgboost import XGBClassifier
        clf = XGBClassifier(
            objective='multi:softprob',
            n_estimators=800,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.8,
            reg_alpha=1e-4,
            reg_lambda=1.0,
            min_child_weight=3,
            gamma=0.0,
            tree_method='hist',
            device = 'cuda',
            random_state=seed,
            n_jobs=-1,
        )
        space = {
            'clf__n_estimators': [400,600,800,1000],
            'clf__max_depth': [4,6,8,10],
            'clf__learning_rate': np.logspace(-2.5, -0.3, 8),
            'clf__subsample': [0.7,0.8,0.9,1.0],
            'clf__colsample_bytree': [0.6,0.7,0.8,0.9,1.0],
            'clf__reg_alpha': np.logspace(-6, -1, 6),
            'clf__reg_lambda': np.logspace(-3, 1, 7),
            'clf__min_child_weight': [1,3,5,7,10],
            'clf__gamma': [0.0, 1e-4, 1e-3, 1e-2],
        }
        return clf, space, 'xgb'

    if mdl == 'et':
        clf = ExtraTreesClassifier(
            n_estimators=700, random_state=seed, n_jobs=-1,
            max_features='sqrt', class_weight='balanced'
        )
        space = {
            'clf__n_estimators': [400,700,1000],
            'clf__max_depth': [None, 8, 12, 16, 24],
            'clf__min_samples_split': [2, 5, 10, 20],
            'clf__min_samples_leaf': [1, 2, 4, 8],
            'clf__max_features': ['sqrt', 'log2', 0.5, None],
            'clf__bootstrap': [False],  # ET suele ir mejor sin bootstrap
        }
        return clf, space, 'et'

    # RF (fallback por defecto)
    clf = RandomForestClassifier(
        n_estimators=500, class_weight='balanced', random_state=seed, n_jobs=-1
    )
    space = {
        'clf__n_estimators': [300,500,700,900],
        'clf__max_depth': [None,8,12,16,24],
        'clf__min_samples_split': [2,5,10,20],
        'clf__min_samples_leaf': [1,2,4,8],
        'clf__max_features': ['sqrt','log2',0.5,None],
        'clf__bootstrap': [True],
    }
    return clf, space, 'rf'

# ========== Threshold tuning (OvR) ==========
def _tune_thresholds_ovr(y_tune: np.ndarray, proba_tune: np.ndarray, n_classes: int,
                         grid: np.ndarray = None) -> np.ndarray:
    if grid is None:
        grid = np.linspace(0.1, 0.9, 41)
    ths = np.zeros(n_classes)
    for c in range(n_classes):
        best_f1, best_t = -1.0, 0.5
        y_bin = (y_tune == c).astype(int)
        p = proba_tune[:, c]
        for t in grid:
            y_hat = (p >= t).astype(int)
            f1 = f1_score(y_bin, y_hat, zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        ths[c] = best_t
    return ths

def _apply_thresholds(proba: np.ndarray, thresholds: np.ndarray) -> np.ndarray:
    scores = proba - thresholds.reshape(1, -1)
    return np.argmax(scores, axis=1)

# ========== Función principal (CV5 + ablations + tuning + guardado) ==========
def run_cv_ablations(
    feats: pd.DataFrame,
    model: str = 'lgbm',                # ahora: 'lgbm' | 'xgb' | 'rf' | 'et'
    feature_set: str = 'all',           # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits: int = 5,
    n_iter: int = 20,
    seed: int = 42,
    tune_thresholds: bool = True,
    save_prefix: str = None,
    save_full: bool = True
) -> Dict[str, Any]:
    X_df, y, le = _prep_feats_labels(feats, feature_set)
    X = X_df.values
    classes = le.classes_
    feature_names = X_df.columns.tolist()
    n_classes = len(classes)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_rows, fold_rows_tuned = [], []
    confusions, confusions_tuned = [], []
    importances_accum = []
    oof_pred = np.full_like(y, fill_value=-1)
    oof_pred_tuned = np.full_like(y, fill_value=-1)
    oof_proba = np.zeros((len(y), n_classes))
    oof_proba_tuned = np.zeros((len(y), n_classes))
    best_fold = {'score': -np.inf, 'params': None}

    with parallel_backend('threading'):
        for k, (tr, te) in enumerate(skf.split(X, y), start=1):
            clf, space, eff_model = _make_model_and_space(model, seed)
            inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
            pipe = Pipeline([('imp', SimpleImputer(strategy='median')), ('clf', clf)])
            search = RandomizedSearchCV(
                pipe, param_distributions=space, n_iter=n_iter, cv=inner_cv,
                scoring='f1_macro', n_jobs=-1, random_state=seed, verbose=0, refit=True
            )
            search.fit(X[tr], y[tr])
            best = search.best_estimator_
            if search.best_score_ > best_fold['score']:
                best_fold = {'score': float(search.best_score_), 'params': search.best_params_}

            # tuning interno (20% del train)
            if tune_thresholds and hasattr(best, "predict_proba"):
                sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
                (tr_sub, tune_sub), = sss.split(X[tr], y[tr])
                tr_abs = np.array(tr)[tr_sub]; tune_abs = np.array(tr)[tune_sub]

                clf2, _, _ = _make_model_and_space(eff_model, seed)
                tuned_pipe = Pipeline([('imp', SimpleImputer(strategy='median')), ('clf', clf2)])
                tuned_pipe.set_params(**{k: v for k, v in search.best_params_.items()})
                tuned_pipe.fit(X[tr_abs], y[tr_abs])

                proba_tune = tuned_pipe.predict_proba(X[tune_abs])
                ths = _tune_thresholds_ovr(y[tune_abs], proba_tune, n_classes)
            else:
                ths = np.full(n_classes, 0.5)

            # evaluación en test fold
            proba_te = best.predict_proba(X[te]) if hasattr(best, "predict_proba") else None
            y_hat = best.predict(X[te])
            f1m = f1_score(y[te], y_hat, average='macro')
            acc = accuracy_score(y[te], y_hat)
            f1_pc = f1_score(y[te], y_hat, average=None, labels=np.arange(n_classes))
            row = {'fold': k, 'f1_macro': f1m, 'accuracy': acc}
            for i, cls in enumerate(classes): row[f'F1_{cls}'] = f1_pc[i]
            fold_rows.append(row)
            cm = confusion_matrix(y[te], y_hat, labels=np.arange(n_classes), normalize='true')
            confusions.append(cm)

            if proba_te is not None and tune_thresholds:
                y_hat_t = _apply_thresholds(proba_te, ths)
                f1m_t = f1_score(y[te], y_hat_t, average='macro')
                acc_t = accuracy_score(y[te], y_hat_t)
                f1_pc_t = f1_score(y[te], y_hat_t, average=None, labels=np.arange(n_classes))
                row_t = {'fold': k, 'f1_macro': f1m_t, 'accuracy': acc_t}
                for i, cls in enumerate(classes): row_t[f'F1_{cls}'] = f1_pc_t[i]
                fold_rows_tuned.append(row_t)
                cm_t = confusion_matrix(y[te], y_hat_t, labels=np.arange(n_classes), normalize='true')
                confusions_tuned.append(cm_t)

            # OOF
            oof_pred[te] = y_hat
            if proba_te is not None: oof_proba[te] = proba_te
            if proba_te is not None and tune_thresholds:
                oof_pred_tuned[te] = y_hat_t
                oof_proba_tuned[te] = proba_te

            # Importancias
            try:
                imp = best.named_steps['clf'].feature_importances_
                if len(imp) == len(feature_names):
                    importances_accum.append(imp)
            except Exception:
                pass

    # Agregados
    cv_df = pd.DataFrame(fold_rows)
    summary = {
        'model_used': eff_model.upper(),
        'feature_set': feature_set,
        'F1_macro_mean': float(cv_df['f1_macro'].mean()),
        'F1_macro_std' : float(cv_df['f1_macro'].std(ddof=1)),
        'accuracy_mean': float(cv_df['accuracy'].mean()),
        'accuracy_std' : float(cv_df['accuracy'].std(ddof=1)),
        'cv_best_score_f1_macro': best_fold['score']
    }
    per_class = []
    for i, cls in enumerate(classes):
        vals = cv_df[f'F1_{cls}']
        per_class.append([cls, float(vals.mean()), float(vals.std(ddof=1))])
    per_class_df = pd.DataFrame(per_class, columns=['class','F1_mean','F1_std']).sort_values('F1_mean', ascending=False)
    mean_cm = np.mean(np.stack(confusions, axis=0), axis=0)

    tuned_available = len(fold_rows_tuned) == n_splits
    cv_tuned_df = pd.DataFrame(fold_rows_tuned) if tuned_available else pd.DataFrame()
    summary_tuned, per_class_tuned_df, mean_cm_tuned = {}, pd.DataFrame(columns=['class','F1_mean','F1_std']), None
    if tuned_available:
        summary_tuned = {
            'F1_macro_mean_tuned': float(cv_tuned_df['f1_macro'].mean()),
            'F1_macro_std_tuned' : float(cv_tuned_df['f1_macro'].std(ddof=1)),
            'accuracy_mean_tuned': float(cv_tuned_df['accuracy'].mean()),
            'accuracy_std_tuned' : float(cv_tuned_df['accuracy'].std(ddof=1)),
        }
        per_class_t = []
        for i, cls in enumerate(classes):
            vals = cv_tuned_df[f'F1_{cls}']
            per_class_t.append([cls, float(vals.mean()), float(vals.std(ddof=1))])
        per_class_tuned_df = pd.DataFrame(per_class_t, columns=['class','F1_mean','F1_std']).sort_values('F1_mean', ascending=False)
        mean_cm_tuned = np.mean(np.stack(confusions_tuned, axis=0), axis=0)

    # Importancias en %
    feat_imp_df = pd.DataFrame(columns=['feature','mean_%','std_%'])
    if importances_accum:
        I = np.vstack(importances_accum)
        mean_imp = I.mean(axis=0)
        std_imp  = I.std(axis=0, ddof=1)
        total = mean_imp.sum() if mean_imp.sum() > 0 else 1.0
        feat_imp_df = pd.DataFrame({
            'feature': feature_names,
            'mean_%': 100.0 * mean_imp / total,
            'std_%' : 100.0 * std_imp  / total
        }).sort_values('mean_%', ascending=False)

    # Guardado
    artifacts = {}
    if save_prefix:
        os.makedirs(os.path.dirname(save_prefix) or ".", exist_ok=True)
        # OOF sin tuning
        oof_df = pd.DataFrame({
            'ID': feats['ID'].astype(str).values if 'ID' in feats.columns else np.arange(len(y)).astype(str),
            'y_true': feats['Classification'].astype(str).values,
            'y_pred': per_class_df['class'].values[0]  # placeholder; sobrescrito abajo
        })
        oof_df['y_pred'] = pd.Series(oof_pred).map(lambda k: le.inverse_transform([k])[0])
        for j, cls in enumerate(classes):
            oof_df[f'proba_{cls}'] = oof_proba[:, j]
        path_oof = f"{save_prefix}_oof.csv"; oof_df.to_csv(path_oof, index=False)
        artifacts['oof_path'] = path_oof

        # OOF tuned
        if tuned_available:
            oof_t_df = pd.DataFrame({
                'ID': oof_df['ID'],
                'y_true': oof_df['y_true'],
                'y_pred_tuned': pd.Series(oof_pred_tuned).map(lambda k: le.inverse_transform([k])[0])
            })
            for j, cls in enumerate(classes):
                oof_t_df[f'proba_{cls}'] = oof_proba_tuned[:, j]
            path_oof_t = f"{save_prefix}_oof_tuned.csv"; oof_t_df.to_csv(path_oof_t, index=False)
            artifacts['oof_tuned_path'] = path_oof_t

        # Folds JSON
        fold_json = {
            'summary': summary, 'summary_tuned': summary_tuned,
            'folds': cv_df.to_dict(orient='records'),
            'folds_tuned': (cv_tuned_df.to_dict(orient='records') if tuned_available else []),
            'classes': classes.tolist(),
            'feature_set': feature_set, 'model': summary['model_used']
        }
        path_folds = f"{save_prefix}_folds.json"
        with open(path_folds, "w", encoding="utf-8") as f: json.dump(fold_json, f, indent=2)
        artifacts['folds_json'] = path_folds

        # Full model con mejores params del mejor fold
        if save_full and best_fold['params'] is not None:
            clf_full, _, eff = _make_model_and_space(summary['model_used'], seed)
            pipe_full = Pipeline([('imp', SimpleImputer(strategy='median')), ('clf', clf_full)])
            pipe_full.set_params(**best_fold['params'])
            pipe_full.fit(X, y)
            path_model = f"{save_prefix}_full_model.joblib"
            joblib_dump(pipe_full, path_model)
            artifacts['full_model_path'] = path_model

            proba_all = pipe_full.predict_proba(X)
            yhat_all = pipe_full.predict(X)
            all_df = pd.DataFrame({
                'ID': feats['ID'].astype(str).values if 'ID' in feats.columns else np.arange(len(y)).astype(str),
                'y_true': feats['Classification'].astype(str).values,
                'y_pred': le.inverse_transform(yhat_all)
            })
            for j, cls in enumerate(classes):
                all_df[f'proba_{cls}'] = proba_all[:, j]
            path_all = f"{save_prefix}_full_all_preds.csv"
            all_df.to_csv(path_all, index=False)
            artifacts['full_all_preds_path'] = path_all

    return {
        'summary': summary,
        'summary_tuned': summary_tuned,
        'per_class': per_class_df,
        'per_class_tuned': per_class_tuned_df,
        'mean_cm': mean_cm,
        'mean_cm_tuned': mean_cm_tuned,
        'feat_importances_%': feat_imp_df,
        'artifacts': artifacts
    }

In [7]:
# Run 1: LGBM con TODAS las features de grafos, thresholds activados
res_all_lgbm = run_cv_ablations(
    feats,
    model='lgbm',
    feature_set='all',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_lgbm_all",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [8]:
# Run 2: LGBM con TODAS las features de grafos, thresholds activados, solo WHVG
res_all_lgbm_whvg = run_cv_ablations(
    feats,
    model='lgbm',
    feature_set='whvg',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_lgbm_all_whvg",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [9]:
# Run 3: RF con TODAS las features de grafos, thresholds activados
res_all_rf = run_cv_ablations(
    feats,
    model='rf',
    feature_set='all',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_rf_all",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [10]:
# Run 4: RF con TODAS las features de grafos, thresholds activados, solo WHVG
res_all_rf_whvg = run_cv_ablations(
    feats,
    model='rf',
    feature_set='whvg',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_rf_all_whvg",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [11]:
# Run 5: XGBoost con TODAS las features de grafos, thresholds activados
res_all_xgb = run_cv_ablations(
    feats,
    model='xgb',
    feature_set='all',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_xgb_all",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [6]:
# Run 6: RF con TODAS las features de grafos, thresholds activados, solo WHVG
res_all_xgb_whvg = run_cv_ablations(
    feats,
    model='xgb',
    feature_set='whvg',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_xgb_all_whvg",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

KeyboardInterrupt: 

In [7]:
# Run 7: ExtraTrees con TODAS las features de grafos, thresholds activados
res_all_et = run_cv_ablations(
    feats,
    model='et',
    feature_set='all',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_et_all",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

In [4]:
# Run 8: ExtraTrees con TODAS las features de grafos, thresholds activados, solo WHVG
res_all_et_whvg = run_cv_ablations(
    feats,
    model='et',
    feature_set='whvg',       # 'hvg' | 'dhvg' | 'whvg' | 'all'
    n_splits=5,
    n_iter=20,
    seed=42,
    tune_thresholds=True,
    save_prefix="vg_cv/run_et_all_whvg",   # carpeta+prefijo (se crean CSV/JSON/MODEL)
    save_full=True
)

## Printing results....

In [15]:
for i in [res_all_lgbm,res_all_lgbm_whvg,res_all_rf,res_all_rf_whvg,res_all_xgb,res_all_xgb_whvg,res_all_et,res_all_et_whvg]:
    # Inspección rápida
    print("== Resumen (sin tuning) ==")
    print(i['summary'])
for i in [res_all_lgbm,res_all_lgbm_whvg,res_all_rf,res_all_rf_whvg,res_all_xgb,res_all_xgb_whvg,res_all_et,res_all_et_whvg]:
    # Inspección rápida
    print("\n== Resumen (tuned) ==")
    print(i['summary_tuned'])

== Resumen (sin tuning) ==
{'model_used': 'LGBM', 'feature_set': 'all', 'F1_macro_mean': 0.6222757623095071, 'F1_macro_std': 0.009848925503454748, 'accuracy_mean': 0.6611891409854057, 'accuracy_std': 0.010120881265417263, 'cv_best_score_f1_macro': 0.6274006061181532}
== Resumen (sin tuning) ==
{'model_used': 'LGBM', 'feature_set': 'whvg', 'F1_macro_mean': 0.6045730338366404, 'F1_macro_std': 0.009762040029579012, 'accuracy_mean': 0.6477148393956542, 'accuracy_std': 0.011942188158526193, 'cv_best_score_f1_macro': 0.6174340114622083}
== Resumen (sin tuning) ==
{'model_used': 'RF', 'feature_set': 'all', 'F1_macro_mean': 0.6051874889168729, 'F1_macro_std': 0.018298600641430705, 'accuracy_mean': 0.6588430999296873, 'accuracy_std': 0.018323616295367173, 'cv_best_score_f1_macro': 0.6149322348204496}
== Resumen (sin tuning) ==
{'model_used': 'RF', 'feature_set': 'whvg', 'F1_macro_mean': 0.605867641520154, 'F1_macro_std': 0.03010911965625042, 'accuracy_mean': 0.6582703092040952, 'accuracy_std': 

In [17]:
res_all_lgbm_whvg['per_class']

,class,F1_mean,F1_std
5,Non-Tr.,0.969022,0.011659
2,CV,0.791020,0.041315
4,HPM,0.756221,0.108801
0,AGN,0.584312,0.031920
6,Other,0.471408,0.047054
3,Flare,0.438745,0.119471
1,Blazar,0.424627,0.043738
7,SN,0.401231,0.059285


In [18]:
pd.DataFrame(res_all_lgbm_whvg['mean_cm'])

,0,1,2,3,4,5,6,7
0,0.636012,0.044246,0.019048,0.085565,0.006349,0.000000,0.003175,0.205605
1,0.158824,0.394118,0.188235,0.058824,0.000000,0.005882,0.047059,0.147059
2,0.038595,0.064502,0.791342,0.017982,0.005128,0.000000,0.033400,0.049051
3,0.249507,0.027833,0.041626,0.437685,0.048768,0.000000,0.006897,0.187685
4,0.120000,0.000000,0.040000,0.080000,0.720000,0.000000,0.000000,0.040000
5,0.020113,0.003390,0.003390,0.003333,0.003333,0.946271,0.000000,0.020169
6,0.111765,0.179739,0.192157,0.011111,0.011111,0.000000,0.403268,0.090850
7,0.305604,0.079614,0.070531,0.097005,0.004444,0.004444,0.017778,0.420580



[Platform 0] NVIDIA CUDA
   [Device 0] NVIDIA GeForce RTX 4070 SUPER
      Type: ALL | GPU
      Max Compute Units: 56
      Global Memory (MB): 12282

[Platform 1] AMD Accelerated Parallel Processing
   [Device 0] gfx1036
      Type: ALL | GPU
      Max Compute Units: 1
      Global Memory (MB): 12263
